<a href="https://colab.research.google.com/github/dfrnks/py-memory-game/blob/linear-network-v2-tpu/learning_tpu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
!python --version

Python 3.7.13


In [15]:
!pip install cloud-tpu-client==0.10 torch==1.11.0 https://storage.googleapis.com/tpu-pytorch/wheels/colab/torch_xla-1.11-cp37-cp37m-linux_x86_64.whl --force-reinstall 


# VERSION = "20220315"  #@param ["1.5" , "20220315", "nightly"]
# !curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
# !python pytorch-xla-env-setup.py --version $VERSION


# VERSION = "1.11"  #@param ["1.11", "nightly", "20220315"]  # or YYYYMMDD format
# !curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
# !python pytorch-xla-env-setup.py --version $VERSION
# import os 
# os.environ['LD_LIBRARY_PATH']='/usr/local/lib'
# !echo $LD_LIBRARY_PATH

# !sudo ln -s /usr/local/lib/libmkl_intel_lp64.so /usr/local/lib/libmkl_intel_lp64.so.1
# !sudo ln -s /usr/local/lib/libmkl_intel_thread.so /usr/local/lib/libmkl_intel_thread.so.1
# !sudo ln -s /usr/local/lib/libmkl_core.so /usr/local/lib/libmkl_core.so.1

# !ldconfig
# !ldd /usr/local/lib/python3.7/dist-packages/torch/lib/libtorch.so

  Using cached https://storage.googleapis.com/tpu-pytorch/wheels/colab/torch_xla-1.11-cp37-cp37m-linux_x86_64.whl (152.9 MB)
  Using cached cloud_tpu_client-0.10-py3-none-any.whl (7.4 kB)
  Using cached torch-1.11.0-cp37-cp37m-manylinux1_x86_64.whl (750.6 MB)
     |████████████████████████████████| 98 kB 3.9 MB/s 
  Using cached google_api_python_client-1.8.0-py3-none-any.whl (57 kB)
     |████████████████████████████████| 156 kB 42.9 MB/s 
     |████████████████████████████████| 93 kB 1.6 MB/s 
     |████████████████████████████████| 96 kB 5.6 MB/s 
     |████████████████████████████████| 503 kB 38.9 MB/s 
     |████████████████████████████████| 241 kB 55.8 MB/s 
     |████████████████████████████████| 40 kB 5.3 MB/s 
     |████████████████████████████████| 1.0 MB 46.8 MB/s 
     |████████████████████████████████| 1.1 MB 42.8 MB/s 
     |████████████████████████████████| 63 kB 1.7 MB/s 
     |████████████████████████████████| 152 kB 46.7 MB/s 
     |████████████████████████████████| 1

In [1]:
%cd /content/
%rm -rf cloned_repo
%rm -rf runs
!git clone -l -s https://github.com/dfrnks/py-memory-game.git cloned_repo
%cd cloned_repo
!git checkout linear-network-v2-tpu
!ls

/content
Cloning into 'cloned_repo'...
remote: Enumerating objects: 318, done.
remote: Counting objects: 100% (318/318), done.
remote: Compressing objects: 100% (228/228), done.
remote: Total 318 (delta 155), reused 239 (delta 87), pack-reused 0
Receiving objects: 100% (318/318), 6.36 MiB | 22.08 MiB/s, done.
Resolving deltas: 100% (155/155), done.
/content/cloned_repo
Branch 'linear-network-v2-tpu' set up to track remote branch 'linear-network-v2-tpu' from 'origin'.
Switched to a new branch 'linear-network-v2-tpu'
checkpoints  LICENSE  play.py	 requirements.txt
learning.py  main.py  README.md  src


In [2]:
import torch

# imports the torch_xla package
import torch_xla
import torch_xla.core.xla_model as xm

# https://colab.research.google.com/github/pytorch/xla/blob/master/contrib/colab/getting-started.ipynb#scrollTo=H9KYz-Vk4fMa

# https://research.google.com/colaboratory/marketplace.html?utm_source=faq&utm_medium=link&utm_campaign=how_long_can_nbs_run

In [3]:
import datetime

from pathlib import Path


from cloned_repo.src import MemoryGameEnv
from cloned_repo.src import MemoryAgent

from cloned_repo import learning

In [4]:
device = xm.xla_device()
#device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
env = MemoryGameEnv((4, 4))

save_dir = Path("checkpoints") / datetime.datetime.now().strftime("%Y-%m-%dT%H-%M-%S")
save_dir.mkdir(parents=True)

agent = MemoryAgent(
    state_dim=(env.action_space.n, env.action_space.n),
    action_dim=env.action_space.n,
    save_dir=save_dir,
    device=device,
    lr=0.001,
    max_memory_size=1000000,
    batch_size=64,
    gamma=0.9,
    exploration_rate=1,
    exploration_rate_decay=0.99999999,
    exploration_rate_min=0.1,
    save_every=5e5,
    burnin=1e4,
    learn_every=3,
    sync_every=1e4
)


learning.run(agent, env, 100)

Training 40:  40%|████      | 40/100 [01:07<01:24,  1.41s/it, Episode 20 - Step 5268 - Epsilon 0.9999473213870391 - Mean Reward -407.476 - Mean Length 250.857 - Mean Loss 0.0 - Mean Q Value 0.0 - Time Delta 33.873 - Time 2022-04-05T13:20:15]

In [ ]:
%load_ext tensorboard
%tensorboard --logdir runs/